In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


iso_code      continent     location        date  total_cases  new_cases  \
175      ABW  North America        Aruba  2020-09-09       2589.0      107.0   
429      AFG           Asia  Afghanistan  2020-09-09      38520.0       26.0   
601      AGO         Africa       Angola  2020-09-09       2981.0       16.0   

     new_cases_smoothed  total_deaths  new_deaths  new_deaths_smoothed  ...  \
175              69.286          15.0         0.0                0.714  ...   
429              45.000        1418.0         3.0                1.714  ...   
601              46.714         120.0         3.0                1.714  ...   

     aged_70_older  gdp_per_capita  extreme_poverty  cardiovasc_death_rate  \
175          7.452       35973.781              NaN                    NaN   
429          1.337        1803.987              NaN                597.029   
601          1.362        5819.495              NaN                276.045   

     diabetes_prevalence  female_smokers  male_smokers  \
175                11.62             NaN           NaN   
429                 9.59             NaN           NaN   
601                 3.94             NaN           NaN   

     handwashing_facilities  hospital_beds_per_thousand  life_expectancy  
175                     NaN                         NaN            76.29  
429                  37.746                         0.5            64.83  
601                  26.664                         NaN            61.15  

[3 rows x 40 columns]

In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

date         country  total_cases_predict  total_cases_real  \
0  2020-09-08          France             335089.0               0.0   
0  2020-09-08           China              90108.0               0.0   
0  2020-09-08           Italy             279965.0               0.0   
0  2020-09-08           Spain             531804.0               0.0   
0  2020-09-08   United States            6391190.0               0.0   
0  2020-09-08           World           27960151.0               0.0   
0  2020-09-08  United Kingdom             352503.0               0.0   
0  2020-09-08         Germany             253092.0               0.0   
0  2020-09-08            Iran             392640.0               0.0   
0  2020-09-08          Turkey             282775.0               0.0   
0  2020-09-08          Brazil            4215135.0               0.0   

   total_deaths_predict  total_deaths_real  error_abs_cases  error_abs_deaths  
0               30749.0                0.0              0.0               0.0  
0                4738.0                0.0              0.0               0.0  
0               35579.0                0.0              0.0               0.0  
0               29534.0                0.0              0.0               0.0  
0              190207.0                0.0              0.0               0.0  
0              902065.0                0.0              0.0               0.0  
0               41590.0                0.0              0.0               0.0  
0                9335.0                0.0              0.0               0.0  
0               22651.0                0.0              0.0               0.0  
0                6772.0                0.0              0.0               0.0  
0              128622.0                0.0              0.0               0.0

In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

date         country  total_cases_predict  total_cases_real  \
0  2020-09-09          France             335089.0          335524.0   
0  2020-09-09           China              90108.0           90087.0   
0  2020-09-09           Italy             279965.0          280153.0   
0  2020-09-08           Spain             531804.0          534513.0   
0  2020-09-09   United States            6391190.0         6327793.0   
0  2020-09-09           World           27960151.0        27609408.0   
0  2020-09-09  United Kingdom             352503.0          352560.0   
0  2020-09-09         Germany             253092.0          253474.0   
0  2020-09-09            Iran             392640.0          391112.0   
0  2020-09-09          Turkey             282775.0          283270.0   
0  2020-09-09          Brazil            4215135.0         4162073.0   

   total_deaths_predict  total_deaths_real  error_abs_cases  error_abs_deaths  
0               30749.0            30764.0           -435.0             -15.0  
0                4738.0             4733.0             21.0               5.0  
0               35579.0            35563.0           -188.0              16.0  
0               29534.0            29594.0          -2709.0             -60.0  
0              190207.0           189679.0          63397.0             528.0  
0              902065.0           898088.0         350743.0            3977.0  
0               41590.0            41586.0            -57.0               4.0  
0                9335.0             9338.0           -382.0              -3.0  
0               22651.0            22542.0           1528.0             109.0  
0                6772.0             6782.0           -495.0             -10.0  
0              128622.0           127464.0          53062.0            1158.0

In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")